# Setup
**Make sure to read all instructions carefully!**

0. [BEFORE UPLOADING ANYTHING](#before-uploading-anything)
1. [Settings](#settings)
2. [Check GPU compatibility](#check-gpu-compatibility)
3. [Download and process .blend (or .zip) file](#download-and-proccess-blend-or-zip-file)
4. [Download and configure Blender](#download-and-configure-blender)
5. [Download rendered images as zip](#download-rendered-images-as-zip)

After you are done configurating, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below.

#### Useful links
[GitHub repository](https://github.com/Nitro4542/colab-render)

[Original GitHub repository by ynshung](https://github.com/ynshung/blender-colab)

## BEFORE UPLOADING ANYTHING
Make sure to include all ressources inside of your .blend file!

`How do I do that?`

For version 4.1:
- Open your project in Blender
- Go to: File -> External Data -> Pack Ressources
- Go to: File -> External Data -> Pack Linked Libraries
- Save your project

## Settings

### General settings

* `Blender_version` : Version of Blender used to render the scene.

___

### Blend file settings

* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.


> #### Upload type
>
> * `direct`: Upload your blender file in the next cell.
>
> * `google_drive`: The blender file will be downloaded directly from Google Drive. You need to specify the path to the blender/zip file at `drive_path`.
>  
> * `url`: Direct link to the blender file in `url_blend`.
>
> * `gdrive_relative`: The Google Drive folder specified at drive_path will be copied directly (as if it's a zipped file).


* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.

* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.

___

### Animation settings

* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.

* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.

___

### Output settings

* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).

* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)

* `zip_files` : Archive multiple animation frames automatically into a zip file.

* `drive_output_path` : Path to your frames/zip file in Google Drive.

___

### Rendering settings

* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.

* `optix_enabled` : Enable OptiX which may boost performance, may be incompatible depending on the version of Blender, project and GPU allocated

In [ ]:
#@markdown ### General settings
blender_version = '4.1.0' #@param ['2.79b', '2.83.20', '2.93.18', '3.3.17', '3.6.10', '4.1.0'] {allow-input: false}
#@markdown ---
#@markdown ### Blend file settings
upload_type = 'url' #@param ['direct', 'google_drive', 'url', 'gdrive_relative'] {allow-input: false}
drive_path = 'path/to/file' #@param {type: 'string'}
url_blend = '' #@param {type: 'string'}
#@markdown ---
#@markdown ### Animation settings
animation = False #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  250#@param {type: 'integer'}
#@markdown ---
#@markdown ### Output settings
download_type = 'gdrive_direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}
#@markdown ---
#@markdown ### Rendering settings
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = True #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}

## Check GPU compatibility

Check if your GPU supports OptiX.

In [ ]:
%cd /content

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

## Download and proccess .blend (or .zip) file

Before you run this part of the code, make sure you have uploaded your Blender Project to your Google Drive and selected it using the code block above.

If you selected to use Google Drive, it will be authorized here.

In [ ]:
import shutil
from google.colab import files, drive
uploaded_filename = ""

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    shutil.copy('/drive/My Drive/' + drive_path, '.')
    uploaded_filename = os.path.basename(drive_path)

!rm -r render
!mkdir render

if upload_type == 'gdrive_relative':
    if not drive_path.endswith('/'):
        drive_path += '/'
    !cp -r '/drive/My Drive/{drive_path}.' 'render/'
elif uploaded_filename.lower().endswith('.zip'):
    !unzip -o $uploaded_filename -d 'render/'
elif uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    raise SystemExit("Invalid file extension, only .blend and .zip can be uploaded.")

## Download and configure Blender

To add your own new version, go to your prefered download mirror
and copy the URL of the desired version linux-x64 tar archive.

If you want to change the download mirror, just change the `download_mirror` variable.

Then, either edit or add the version below and also at the version selection at the top.

This cell will also reinstall a library which is needed for Blender to run.

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4

os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

download_mirror = "https://ftp.nluug.nl/pub/graphics/blender/release"
blender_url_dict = {'2.79b':"{download_mirror}/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
                    '2.83.20':"{download_mirror}/release/Blender2.83/blender-2.83.20-linux-x64.tar.xz",
                    '2.93.18':"{download_mirror}/release/Blender2.93/blender-2.93.18-linux-x64.tar.xz",
                    '3.3.17':"{download_mirror}/release/Blender3.3/blender-3.3.17-linux-x64.tar.xz",
                    '3.6.10':"{download_mirror}/release/Blender3.6/blender-3.6.10-linux-x64.tar.xz",
                    '4.1.0':"{download_mirror}/release/Blender4.1/blender-4.1.0-linux-x64.tar.xz"}

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (example: Tesla K80) is not supported and you need to switch back to Cycles.")
    renderer = "OPTIX"

## Start rendering

All of the rendered frames will be saved to your Google Drive.

If you want to download them all in a compressed zip file, take a look at [the next step](#download-rendered-images-as-zip).

In [ ]:
%cd /content

!rm -r output
!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

if download_type != 'gdrive_direct':
    output_path = '/content/output/' + output_name
else:
    output_path = '/drive/My Drive/' + drive_output_path + output_name

%cd /content/$blender_version

if animation:
    if start_frame == end_frame:
        !./$blender_version/blender -b '/content/render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./$blender_version/blender -b '/content/render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./$blender_version/blender -b '/content/render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"

## Download rendered images as zip

All images are saved in your Google Drive folder you specified above, but if you want to get them in a compressed format, run the code below.

In [ ]:
%cd /content

path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if download_type == 'gdrive_direct':
    pass
elif len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    if download_type == 'direct':
        files.download('output/' + files_folder[0])
    else:
        shutil.copy('/content/' + render_img, '/drive/My Drive/' + drive_output_path)
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
    if download_type == 'direct':
        files.download(output_folder_name + '.zip')
    else:
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/My Drive/' + drive_output_path)
elif download_type == 'direct':
    for f in files_folder:
        files.download('output/{}'.format(f))
    # Drive, no zip
    else:
        for f in files_folder:
          shutil.copy("/content/output/" + f, '/drive/My Drive/' + drive_output_path + f)
else:
    raise SystemExit("No frames were rendered.")

## Disclaimer
Google Colab is targeted to researchers and students to run AI/ML tasks, data analysis and education, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies time by time. [Colab Pro and Colab Pro+](https://colab.research.google.com/signup) are available for those who wanted to have more powerful GPU and longer runtimes for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info. In some cases, it might be faster to use an online Blender renderfarm.